In [1]:
from dask_gateway import Gateway

gateway = Gateway()
clusters = gateway.list_clusters()

/srv/conda/envs/notebook/lib/python3.9/site-packages/dask_gateway/client.py:21: FutureWarning: format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.
  from distributed.utils import LoopRunner, format_bytes


In [2]:
clusters

[ClusterReport<name=default.f57301bfcf744b4d8711c8c7a0cef6fe, status=RUNNING>]

In [3]:
cluster = gateway.connect(clusters[0].name)

In [4]:
client = cluster.get_client()

In [79]:
#ts = client.get_task_stream()

In [195]:
ts = merge(ts, client.get_task_stream())

In [197]:
def merge(ts1, ts2):
    last = ts1[-1]["startstops"][0]["stop"]
    last_index, = [i for i, t in enumerate(ts2) if t["startstops"][0]["stop"] == last]
    return ts1 + ts2[last_index:]

In [198]:
len(ts)

266039

```
import pickle
with open("task_stream.pickle", "wb") as of:
    pickle.dump(ts, of)
```

In [190]:
import pickle
import gzip
with gzip.open("task_stream.pickle.gz", "wb") as of:
    of.write(pickle.dumps(ts))

Alternative to plot from json task stream:

In [191]:
from distributed.dashboard.components.scheduler import task_stream_figure
from distributed.diagnostics.task_stream import rectangles
from bokeh.plotting import show, output_file, save
from bokeh.io import output_notebook

In [192]:
filename = "taskstream.html"
last = max([t["startstops"][0]["stop"] for t in ts if "startstops" in t])
#rects = rectangles([t for t in ts if "startstops" in t and t["startstops"][0]["start"] > last - 600])
rects = rectangles(ts)
source, figure = task_stream_figure(sizing_mode="stretch_both", min_height=500)
source.data.update(rects)
output_file(filename=filename, title="Dask Task Stream")
save(figure, filename=filename)

'/home/jovyan/analysis-code/notebooks/taskstream_4l.html'

In [193]:
f"""{(ts[-1]["startstops"][-1]["stop"] - ts[0]["startstops"][0]["start"]) / 60:.2f} minutes"""

'75.43 minutes'